#Domain ADAPTATIVE PRETRAINING:  
In this notebook, we preprocess the data from the RAW papers, that we are going to use to train the DAPT.
AND we train the DAPT to the answerdotai/ModernBERT (base modernBERT)

Note, to run this model you need GPU A100 4hours. In the preprocesing part of the latex might give an error.

In [ ]:
!pip install -U bitsandbytes
!pip install -U accelerate transformers peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
#libraries:
from datasets import load_dataset
from transformers import BertTokenizer

import os, tarfile, gzip, re, json, glob, pathlib, warnings
from tqdm import tqdm
#from pylatexenc.latex2text import LatexNodes2Text
import spacy
from spacy.language import Language
import pandas as pd, json, pathlib
from datasets import Dataset
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoModel, BitsAndBytesConfig, TrainingArguments, Trainer




In [ ]:
!pwd

/content


## Prepare the domain corpus, (unlabeled)

I will do some cleaning of the mathematical papers, where instead of focusing on theorems, I want to focus on the content of the document.



In [ ]:
import zipfile

with zipfile.ZipFile(SRC_DIR + '/arxiv_2502.zip', 'r') as zip_ref:
    zip_ref.extractall(OUT_DIR)  # Extract to a folder

In [ ]:
import re
import tarfile
import pandas as pd


def extract_relevant_sections(text: str) -> str:
    """
    Extract the relevant parts of the LATEX Paper
    -  Abstract
    - Introduction and Conclusions
    -  Definition, Theorem, Lemma, Corollary y Proof
    """
    start_match = re.search(
        r"\\begin\{abstract\}|^\\section\*?\{(?:Introduction|)\}",
        text,
        flags=re.S | re.M | re.I
    )
    if start_match:
        text = text[start_match.start():]

    parts = []
    # Abstract
    m = re.search(r"\\begin\{abstract\}(.+?)\\end\{abstract\}", text, flags=re.S | re.I)
    if m:
        parts.append(m.group(1))
    # Wanted Sections
    sec_split = re.split(r'(?m)^\\section\*?\{(.+?)\}', text)
    for i in range(1, len(sec_split), 2):
        title = sec_split[i].strip().lower()
        content = sec_split[i+1]
        if any(k in title for k in ['introduction', 'methodology', 'conclusion', 'model', 'results','proves','proof','models']):
            parts.append(content)
    for env in ['definition', 'theorem', 'lemma', 'corollary', 'proof']:
        for m in re.findall(rf"\\begin\{{{env}\}}(.+?)\\end\{{{env}\}}", text, flags=re.S | re.I):
            parts.append(m)
    return '\n'.join(parts)


def clean_latex(text: str) -> str:
    """
    Clean the not needed latex parts.
    """
    #  structure: \begin{...}, \end{...} and \item
    text = re.sub(r"\\(?:begin|end)\{[^}]+\}", '', text)
    text = re.sub(r"\\item\b", '', text)
    text = re.sub(r"\\(?:noindent|medskip|smallskip|bigskip|newpage|clearpage)\b", "\n", text)


    # Other extra things to eliminate
    text = re.sub(r"\\(?:chapter|section|subsection|subsubsection|paragraph|subparagraph)\*?\{[^}]*\}", ' ', text)
    text = re.sub(r"\\(?:label|ref|eqref|pageref)\{[^}]*\}", '', text)
    text = re.sub(r"\\cite[t|p|alp]?\{[^}]*\}", '', text)
    text = re.sub(r"\\bibliography\{[^}]*\}", '', text)
    text = re.sub(r"\\bibliographystyle\{[^}]*\}", '', text)
    text = re.sub(r"\\footnote\{([^}]*)\}", r"\1", text)
    text = re.sub(r"\\href\{[^}]*\}\{([^}]*)\}", r"\1", text)
    text = re.sub(r"\\url\{([^}]*)\}", r"\1", text)
    text = re.sub(r"%.*", '', text)

    # Normalize  math-mode to $...$
    text = re.sub(r"\$\$(.+?)\$\$", r"$\1$", text, flags=re.S)
    text = re.sub(r"\\\[(.+?)\\\]",    r"$\1$", text, flags=re.S)
    text = re.sub(r"\\\((.+?)\\\)",    r"$\1$", text, flags=re.S)

    # puntuation outside math-mode
    def _fix_punct(m): return f'${m.group(1)}$'+m.group(2)
    text = re.sub(r"\$(.+?)([.,;:])\$", _fix_punct, text, flags=re.S)

    # process what is outside math-mode
    parts = re.split(r"(\$.+?\$)", text, flags=re.S)
    for idx, seg in enumerate(parts):
        if not seg.startswith('$'):
            seg = seg.replace('!', 'clik')
            seg = re.sub(r"\\[a-zA-Z]+\*?(?:\[[^\]]*\])?\{([^}]*)\}", r"\1", seg)
            seg = re.sub(r"\s*-\s*", ' - ', seg)
            seg = re.sub(r"\s+", ' ', seg)
            parts[idx] = seg
    return ''.join(parts).strip()


def extract_and_clean_records(archive_path: str):
    """
    Main function.
    Open each file and extract the relevant parts.
    """
    records = []
    paper_name = os.path.basename(archive_path)

    with tarfile.open(archive_path, 'r:gz') as tar:
        members = [m for m in tar.getmembers() if m.name.endswith('.tex')]
        if not members:
            members = tar.getmembers()

        for m in members:
            try:
                f = tar.extractfile(m)
                if not f:
                    continue
                raw = f.read().decode('utf-8', errors='ignore')

                # we call other previous functions
                relevant = extract_relevant_sections(raw)
                source   = relevant if relevant.strip() else raw

                # clean
                cleaned  = clean_latex(source)

                records.append({
                    'paper': paper_name,
                    'clean_text': cleaned
                })
            except Exception as e:
                print(f" Error procesing {m.name} in {paper_name}: {e}")
    return records


In [ ]:
#we add all the 3 months into a single CSV
OUT_DIR = ['/content/drive/MyDrive/Mater Thesis/MSThesis/PREPROCESSING/DATA/01', '/content/drive/MyDrive/Mater Thesis/MSThesis/PREPROCESSING/DATA/02','/content/drive/MyDrive/Mater Thesis/MSThesis/PREPROCESSING/DATA/03']  # ajusta tus rutas
all_records = []

for folder in OUT_DIR:
    for fn in os.listdir(folder):

        if fn.lower().endswith('.gz'):
            path = os.path.join(folder, fn)
            recs = extract_and_clean_records(path)
            all_records.extend(recs)
        elif fn.lower().endswith('.tex'):
            full = os.path.join(folder, fn)
            with open(full, 'r', encoding='utf-8', errors='ignore') as f:
                raw = f.read()
            relevant = extract_relevant_sections(raw)
            source   = relevant if relevant.strip() else raw
            cleaned  = clean_latex(source)
            all_records.append({
                'paper': fn,
                'clean_text': cleaned
            })

In [ ]:
len(all_records)

WE save the data.

In [ ]:
#
df = pd.DataFrame(all_records)
csv_path = '/content/drive/MyDrive/Mater Thesis/MSThesis/PREPROCESSING/DATA/all_cleaned_papers2.csv'
df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"✅ CSV con {len(df)} filas creado en “{csv_path}”")


In [ ]:
csv_path =''
data.to_csv(csv_path, index=False, encoding='utf-8')
print(f"✅ CSV con {len(data)} filas creado en “{csv_path}”")

Download the data

In [ ]:

data = pd.read_csv('/content/drive/Shareddrives/Master_Thesis/DAPT/all_cleaned_papers2.csv')
data.head()

In [ ]:
#eliminate duplicates
data['__len'] = data['clean_text'].str.len()


data = data.sort_values('__len', ascending=False)
data_unique = data.drop_duplicates(subset='paper', keep='first')

data_unique = data_unique.drop(columns='__len')

data = data_unique.reset_index(drop=True)

print("Filas tras eliminar duplicados:", len(data))

Filas tras eliminar duplicados: 12099


In [ ]:
data['paper'].value_counts()

,count
paper,
2503.22907.gz,1
2503.21909.gz,1
2503.00438.gz,1
2502.12760.gz,1
2502.08497.gz,1
...,...
2501.16223.gz,1
2501.06015.gz,1
2501.12609.gz,1


### Tokenize the Corpus:

In [ ]:
from huggingface_hub import notebook_login, HfApi
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)
import pandas as pd

# Log in and create a new repo for this fine-tuned ModernBERT model
notebook_login()
api = HfApi()
api.create_repo("Master-thesis-NAP/ModernBert-DAPT-math", exist_ok=True)

RepoUrl('https://huggingface.co/Master-thesis-NAP/ModernBert-DAPT-math', endpoint='https://huggingface.co', repo_type='model', repo_id='Master-thesis-NAP/ModernBert-DAPT-math')

In [ ]:
from transformers import (
    BertTokenizerFast,
    BertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset, Dataset
import pandas as pd

modelo_base = "answerdotai/ModernBERT-base"


import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


---

---



In [ ]:
data = data.rename(columns={"clean_text": "text"})
data = data.dropna(subset=["text"])
data["text"] = data["text"].astype(str)

dataset = Dataset.from_pandas(data)

tokenizer = AutoTokenizer.from_pretrained(modelo_base)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        return_overflowing_tokens=True, #this
        stride=126, #this
        padding="max_length",
        max_length=1024,
    )
    result = {k: [] for k in tokenized.keys() if k != "overflow_to_sample_mapping"}
    for i in range(len(tokenized["input_ids"])):
        for k in result:
            result[k].append(tokenized[k][i])
    return result

tokenized = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names ,
)

Map:   0%|          | 0/12096 [00:00<?, ? examples/s]

In [ ]:
# Remove columns that we dont need
candidatas = ["__index_level_0__", "text", "paper"]

cols_presentes = [c for c in candidatas if c in tokenized.column_names]


if cols_presentes:
    tokenized = tokenized.remove_columns(cols_presentes)


In [ ]:
# Data collator for MLM
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,
    pad_to_multiple_of=8, # ensure uniform tensor shapes for optimal batching.
)


training_args = TrainingArguments( #define training arguments
    output_dir="/content/drive/Shareddrives/Master_Thesis/DAPT/Models/dapt-modernbert-adapted",
    overwrite_output_dir=True,

    per_device_train_batch_size = 6,
    gradient_accumulation_steps = 4,
    learning_rate=2e-5,
    warmup_ratio=0.06,
    weight_decay=0.01,
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=False,
    logging_steps=1000,
    save_steps=2000,
    save_total_limit=2,
)

# download model
model = AutoModelForMaskedLM.from_pretrained("answerdotai/ModernBERT-base")
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized,
)

#train the model
trainer.train()

output_dir='/content/drive/Shareddrives/Master_Thesis/DAPT/Models/dapt-modernbert-adapted2'
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

#upload to hugging-face
model.push_to_hub("Master-thesis-NAP/ModernBert-DAPT-math")
tokenizer.push_to_hub("Master-thesis-NAP/ModernBert-DAPT-math")


Step,Training Loss
1000,2.032100
2000,1.979400
3000,1.930100
4000,1.943400
5000,1.908400
6000,1.905100
7000,1.879800
8000,1.866500
9000,1.883100
10000,1.855800


model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Master-thesis-NAP/ModernBert-DAPT-math/commit/030ce154a6b3479b4a4a6ce3a5e5fdeea691675c', commit_message='Upload tokenizer', commit_description='', oid='030ce154a6b3479b4a4a6ce3a5e5fdeea691675c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Master-thesis-NAP/ModernBert-DAPT-math', endpoint='https://huggingface.co', repo_type='model', repo_id='Master-thesis-NAP/ModernBert-DAPT-math'), pr_revision=None, pr_num=None)

In [ ]:
#save in google drive and push to hugging face for later use
output_dir='/content/drive/Shareddrives/Master_Thesis/DAPT/Models/dapt-modernbert-adapted'
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

model.push_to_hub("Master-thesis-NAP/ModernBert-DAPT-math")
tokenizer.push_to_hub("Master-thesis-NAP/ModernBert-DAPT-math")

Test to load the model from hugging face


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("Master-thesis-NAP/ModernBert-DAPT-math")
model     = AutoModelForMaskedLM.from_pretrained("Master-thesis-NAP/ModernBert-DAPT-math")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Check if the weights have been updated

In [ ]:
basemodel = AutoModelForMaskedLM.from_pretrained("answerdotai/ModernBERT-base")
model     = AutoModelForMaskedLM.from_pretrained("Master-thesis-NAP/ModernBert-DAPT-math")
import torch

def models_have_same_weights(model1, model2):
    for p1, p2 in zip(model1.parameters(), model2.parameters()):
        if not torch.equal(p1, p2):
            return False
    return True

same = models_have_same_weights(basemodel, model)
print("Same weights:", same)

Same weights: False
